# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/prsrivastava/DSI_Demo/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [9]:
from sklearn.model_selection import train_test_split
# Create features (X) and target (Y)
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

# Split the data into training and testing sets (70-30 split, random state 42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Output the shapes to confirm the split
print(f"Training set: X_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"Testing set: X_test: {X_test.shape}, Y_test: {Y_test.shape}")

Training set: X_train: (22792, 14), Y_train: (22792,)
Testing set: X_test: (9769, 14), Y_test: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

In the code, the random state is set to 42.
The random state ensures that the data split is reproducible. When working on machine learning projects, researchers or developers may need to replicate results for debugging, comparison, or collaborative purposes. Setting a random state allows the split to be deterministic so that every time the code is run, the exact same training and testing sets are created.
This consistency is essential for benchmarking models and ensuring reliable comparisons between different approaches or parameter settings.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline  # Import Pipeline

# Identify numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship',
                        'race', 'sex', 'native-country']

# Define the transformations for numerical variables
numerical_transformer = Pipeline(steps=[
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN-based imputation
    ('RobustScaler', RobustScaler())  # Scaling using statistics 
])

# Define the transformations for categorical variables
categorical_transformer = Pipeline(steps=[
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),  # Simple imputation
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encoding with binary drop
])

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),  # Custom label for numerical transformations
        ('cat_transforms', categorical_transformer, categorical_features)  # Custom label for categorical transformations
    ]
)

print(preprocessor)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                            

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)

# Create the full pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Add the ColumnTransformer as a preprocessing step
    ('classifier', classifier)  # Add RandomForestClassifier as the final step
])

# Output the pipeline structure
print(model_pipeline)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                              

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [14]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',  # Negative log loss
    'roc_auc': 'roc_auc',  # ROC AUC
    'accuracy': 'accuracy',  # Accuracy
    'balanced_accuracy': make_scorer(balanced_accuracy_score)  # Balanced accuracy
}

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline,
    X_train,
    Y_train,
    cv=5,
    scoring=scoring,
    return_train_score=True
)

# Display training and validation results
print("Cross-validation Results:")
print(cv_results)


c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Cross-validation Results:
{'fit_time': array([7.93030119, 8.03047729, 7.94323897, 7.95538592, 7.92819071]), 'score_time': array([0.16977119, 0.16346836, 0.15550447, 0.15577674, 0.16130328]), 'test_neg_log_loss': array([-0.35767483, -0.36923923, -0.37598795, -0.35679093, -0.3803792 ]), 'train_neg_log_loss': array([-0.08118902, -0.08151629, -0.08146857, -0.08251057, -0.08136767]), 'test_roc_auc': array([0.90438406, 0.90107902, 0.90137791, 0.90724978, 0.9022994 ]), 'train_roc_auc': array([1., 1., 1., 1., 1.]), 'test_accuracy': array([0.85062514, 0.85040579, 0.85410268, 0.85980693, 0.85607723]), 'train_accuracy': array([0.99994515, 1.        , 0.99994516, 1.        , 1.        ]), 'test_balanced_accuracy': array([0.7744843 , 0.77188128, 0.77601679, 0.7828585 , 0.77608926]), 'train_balanced_accuracy': array([0.9998869 , 1.        , 0.99988693, 1.        , 1.        ])}


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [15]:
import pandas as pd

# Convert results to DataFrame
cv_df = pd.DataFrame(cv_results)

# Rename columns for clarity
cv_df = cv_df.rename(columns={
    'train_neg_log_loss': 'train_neg_log_loss',
    'test_neg_log_loss': 'val_neg_log_loss',
    'train_roc_auc': 'train_roc_auc',
    'test_roc_auc': 'val_roc_auc',
    'train_accuracy': 'train_accuracy',
    'test_accuracy': 'val_accuracy',
    'train_balanced_accuracy': 'train_balanced_accuracy',
    'test_balanced_accuracy': 'val_balanced_accuracy',
})

# Sort by negative log loss (validation set)
cv_df_sorted = cv_df.sort_values(by='val_neg_log_loss', ascending=False)
print("\nFold-level Results Sorted by Validation Negative Log Loss:")
print(cv_df_sorted)



Fold-level Results Sorted by Validation Negative Log Loss:
   fit_time  score_time  val_neg_log_loss  train_neg_log_loss  val_roc_auc  \
3  7.955386    0.155777         -0.356791           -0.082511     0.907250   
0  7.930301    0.169771         -0.357675           -0.081189     0.904384   
1  8.030477    0.163468         -0.369239           -0.081516     0.901079   
2  7.943239    0.155504         -0.375988           -0.081469     0.901378   
4  7.928191    0.161303         -0.380379           -0.081368     0.902299   

   train_roc_auc  val_accuracy  train_accuracy  val_balanced_accuracy  \
3            1.0      0.859807        1.000000               0.782859   
0            1.0      0.850625        0.999945               0.774484   
1            1.0      0.850406        1.000000               0.771881   
2            1.0      0.854103        0.999945               0.776017   
4            1.0      0.856077        1.000000               0.776089   

   train_balanced_accuracy  
3  

Calculate the mean of each metric. 

In [16]:
# Calculate mean values of each metric
mean_results = cv_df.mean()
print("\nMean Performance Metrics Across Folds:")
print(mean_results)



Mean Performance Metrics Across Folds:
fit_time                   7.957519
score_time                 0.161165
val_neg_log_loss          -0.368014
train_neg_log_loss        -0.081610
val_roc_auc                0.903278
train_roc_auc              1.000000
val_accuracy               0.854204
train_accuracy             0.999978
val_balanced_accuracy      0.776266
train_balanced_accuracy    0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [18]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the pipeline to the training data
model_pipeline.fit(X_train, Y_train)

# Generate predictions and prediction probabilities
Y_pred_proba = model_pipeline.predict_proba(X_test)
Y_pred = model_pipeline.predict(X_test)

# Calculate metrics on testing data
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),  # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # ROC AUC (using probabilities for positive class)
    'accuracy': accuracy_score(Y_test, Y_pred),  # Accuracy
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)  # Balanced accuracy
}

# Display test metrics
print("\nPerformance Metrics on Testing Data:")
print(test_metrics)



Performance Metrics on Testing Data:
{'neg_log_loss': -0.39726506589398913, 'roc_auc': 0.8999828704291436, 'accuracy': 0.8551540587572934, 'balanced_accuracy': 0.7751631656838177}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

It is convenient for a streamlines workflow, making smoother compatibility with ML algorithms, reduction or manual errors by incorporation the transformatuon directly into the data loading pipleline and hence increase of the overall efficiency  during data loading. Having the target variable in a numeric format makes it easier to compute statistics, visualize data distributions, and analyze correlations with other variables.

In summary, recoding income during data loading ensures a seamless, efficient, and error-free workflow for subsequent data processing and machine learning tasks.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.